# ✅ STEP 1: Install required packages

In [1]:
!pip install yfinance pandas numpy scikit-learn matplotlib tensorflow joblib --quiet

# ✅ STEP 2: Import libraries

In [8]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import tensorflow as tf
import datetime
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler


# ✅ STEP 3: Load stock data

In [9]:
ticker = "AAPL"
start_date = "2015-01-01"
# end_date = "2024-01-01"
end_date = datetime.datetime.today().strftime('%Y-%m-%d')

data = yf.download(ticker, start=start_date, end=end_date)
data = data[["Close"]]
data.dropna(inplace=True)
data.tail()


/tmp/ipython-input-9-2425104613.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed


Price,Close
Ticker,AAPL
Date,
2025-06-16,198.419998
2025-06-17,195.639999
2025-06-18,196.580002
2025-06-20,201.000000
2025-06-23,201.500000


# ✅ STEP 4: Scale and prepare data

In [10]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

# Save scaler
joblib.dump(scaler, "scaler.gz")

def create_sequences(data, seq_len=60):
    x, y = [], []
    for i in range(seq_len, len(data)):
        x.append(data[i-seq_len:i])
        y.append(data[i])
    return np.array(x), np.array(y)

sequence_length = 60
X, y = create_sequences(scaled_data, sequence_length)
X = X.reshape((X.shape[0], X.shape[1], 1))


# ✅ STEP 5: Build and train LSTM model

In [11]:
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X.shape[1], 1)),
    LSTM(50),
    Dense(1)
])
model.compile(optimizer="adam", loss="mean_squared_error")
model.fit(X, y, epochs=10, batch_size=32)

# Save the model
model.save("lstm_model.h5")


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


81/81 ━━━━━━━━━━━━━━━━━━━━ 5s 38ms/step - loss: 0.0307
Epoch 2/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - loss: 4.6284e-04
Epoch 3/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - loss: 5.0574e-04
Epoch 4/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - loss: 4.5161e-04
Epoch 5/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - loss: 4.5502e-04
Epoch 6/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - loss: 4.2562e-04
Epoch 7/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - loss: 4.3712e-04
Epoch 8/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - loss: 4.1624e-04
Epoch 9/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - loss: 4.5046e-04
Epoch 10/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - loss: 3.5779e-04


# ✅ STEP 6: Predict future prices (7, 15, 30 days)

In [12]:
forecast_days = [7, 15, 30]
last_sequence = scaled_data[-sequence_length:]
future_preds = {}

for days in forecast_days:
    temp_seq = last_sequence.copy()
    preds = []
    for _ in range(days):
        pred = model.predict(temp_seq.reshape(1, sequence_length, 1), verbose=0)[0][0]
        preds.append(pred)
        temp_seq = np.append(temp_seq[1:], [[pred]], axis=0)
    future_preds[days] = scaler.inverse_transform(np.array(preds).reshape(-1, 1)).flatten()

for days, preds in future_preds.items():
    print(f"\n📈 Future {days}-day prediction:")
    print(preds)



📈 Future 7-day prediction:
[200.80951 200.97386 201.19788 201.45758 201.73895 202.03365 202.33665]

📈 Future 15-day prediction:
[200.80951 200.97386 201.19788 201.45758 201.73895 202.03365 202.33665
 202.64485 202.95644 203.27011 203.58495 203.90048 204.21631 204.53206
 204.84756]

📈 Future 30-day prediction:
[200.80951 200.97386 201.19788 201.45758 201.73895 202.03365 202.33665
 202.64485 202.95644 203.27011 203.58495 203.90048 204.21631 204.53206
 204.84756 205.1625  205.47678 205.79019 206.10263 206.41394 206.72409
 207.03294 207.34052 207.64664 207.9514  208.25461 208.55632 208.85652
 209.15527 209.4523 ]


# ✅ STEP 7: Download model files in Colab

In [13]:
from google.colab import files
files.download("lstm_model.h5")
files.download("scaler.gz")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>